# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [8]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [9]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [10]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    X_train = X_train.copy()
    X_val = X_val.copy()
    scaler = None
    pca = None
    
    if hyperparams['use_pca']:
        #standardizzazione i dati prima della PCA
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)  
        X_val = scaler.transform(X_val)

        #PCA
        pca = PCA(n_components=hyperparams['pca_components'])
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)

    
    if hyperparams['data_standardize']:
        #standardizzazione dei dati
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Aggiunge il termine costante ai dati e
    # Concatenare una colonna di uno alla matrice delle features
    X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_val = np.c_[np.ones((X_val.shape[0], 1)), X_val]
    

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        # Regolarizzazione L2
        lambda_ = hyperparams['reg_lambda']  # parametro di regolarizzazione

        X_transposta = X_train.T   # Trasposta della matrice X
        X_transposta_X = X_transposta @ X_train   # Prodotto X^T * X

        # Aggiungo la regolarizzazione L2: lambda * I
        I = np.eye(X_transposta_X.shape[1]) # matrice identità
        I[0, 0] = 0
        regularized_matrix = X_transposta_X + (lambda_ * I)  # ((X^T * X) + l * I)

        # Calcolo l'inversa
        inversa = np.linalg.inv(regularized_matrix)  # ((X^T * X) + l * I)^-1
        
        #trovo Theta della regolarizzazione L2
        parametro = inversa @ X_transposta @ y_train  # ((X^T * X) + l * I)^-1 * X^T * y
        
    else:
        # Regressione Lineare
        X_transpose = X_train.T  # Trasposta della matrice X
        X_transpose_X = X_transpose @ X_train  # Prodotto X^T * X
        X_transpose_X_inv = np.linalg.inv(X_transpose_X)  # Inversa di (X^T * X)
        X_transpose_X_inv_X_transpose = X_transpose_X_inv @ X_transpose  # Prodotto (X^T * X)^-1 * X^T

        #trovo Theta della regressione lineare
        parametro = X_transpose_X_inv_X_transpose @ y_train  # (X^T * X)^-1 * X^T * y

    # Calcolo predizioni
    #...
    y_train_pred = X_train @ parametro  # Predizioni sui dati di training
    y_val_pred = X_val @ parametro  # Predizioni sui dati di validazione

    # Calcola il MAE
    # ...
    mae_train = np.mean(np.abs(y_train - y_train_pred))
    mae_val = np.mean(np.abs(y_val - y_val_pred ))
        
    return mae_train, mae_val, parametro, scaler, pca

# Dividi il dataset in training e test set
# ...
train_set = 0.8
test_set = 0.2

# svolgimento...

num_train = int(train_set * X.shape[0])
X_train = X[:num_train]
y_train = y[:num_train]

X_test = X[num_train:]
y_test = y[num_train:]

print(f"Training set: {X_train.shape} osservazioni")
print(f"Test set: {X_test.shape} osservazioni")

# Dividi il training set in training set effettivo e validation set
# ...
train_set_effettivo = 0.8 * train_set
val_set = 0.2 * train_set
# svolgimento...
num_train = int(train_set_effettivo * X.shape[0])
num_val = int(val_set * X.shape[0])

X_train_effet = X[:num_train]
y_train_effet = y[:num_train]

X_valid = X[num_train:num_train + num_val]
y_valid = y[num_train:num_train + num_val]

print(f"Training set: {X_train_effet.shape} osservazioni")
print(f"Validation set: {X_valid.shape} osservazioni")

# Trova la configurazione di iperparametri migliore
# configurazioni possibili 
configs = []
for use_pca, data_standardize, use_reg in itertools.product([True, False], repeat=3):
    if use_pca and not data_standardize:
        continue  # PCA richiede dati standardizzati
    for reg_lambda in ([None] if not use_reg else [0.01, 0.1, 1]):
        for pca_components in ([None] if not use_pca else [5, 8, 11]):
            configs.append({
                'use_pca': use_pca,
                'pca_components': pca_components,
                'data_standardize': data_standardize,
                'use_regularization': use_reg,
                'reg_lambda': reg_lambda
            })

# Ricerca della miglior configurazione
best_config = None
best_val_mae = float('inf')
best_train_mae = None
best_scaler = None
best_pca = None
best_theta = None

for values in configs:
    # Riallena il modello sul training set completo
    mae_val, mae_train, parametro, scaler, pca = pipeline(X_train, y_train, X_valid, y_valid, values)
    if mae_val < best_val_mae:
        best_val_mae = mae_val 
        best_train_mae = mae_train
        best_config = values
        best_scaler = scaler
        best_pca = pca
        best_theta = parametro

        
print("Miglior configurazione trovata:", best_config)
print("MAE Validation:", best_val_mae)
print("MAE Training:", best_train_mae)

# Calcola il MAE sul test set
# Ricalcolo sul training completo + test finale
X_trainval_copy = X_train.copy()
X_test_copy = X_test.copy()

if best_config['use_pca']:
    X_trainval_copy = best_scaler.fit_transform(X_trainval_copy)
    X_test_copy = best_scaler.transform(X_test_copy)

    X_trainval_copy = best_pca.fit_transform(X_trainval_copy)
    X_test_copy = best_pca.transform(X_test_copy)

if best_config['data_standardize']:
    X_trainval_copy = best_scaler.fit_transform(X_trainval_copy)
    X_test_copy = best_scaler.transform(X_test_copy)
    
X_trainval_copy = np.c_[np.ones((X_trainval_copy.shape[0], 1)), X_trainval_copy]
X_test_copy = np.c_[np.ones((X_test_copy.shape[0], 1)), X_test_copy]

# Calcolo le predizioni sul test set
y_test_pred = X_test_copy @ best_theta

# Calcola il MAE sul test set
mae_test = np.mean(np.abs(y_test - y_test_pred))

# Stampa  risultati
# ...
print("MAE Test:", mae_test)

Training set: (3918, 11) osservazioni
Test set: (980, 11) osservazioni
Training set: (3134, 11) osservazioni
Validation set: (783, 11) osservazioni


/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Miglior configurazione trovata: {'use_pca': False, 'pca_components': None, 'data_standardize': True, 'use_regularization': False, 'reg_lambda': None}
MAE Validation: 0.5942410518368386
MAE Training: 0.5686842321646448
MAE Test: 0.5513533205470418
